In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('../Recommender/webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [23]:
def weighted_random_recommendation(data, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
    items_weights.sort_values('sales_value', ascending=False, inplace=True)

    items_weights['weight'] = items_weights.sales_value / items_weights.sales_value.sum()
    items_weights.drop(columns='sales_value', inplace=True)

    items = items_weights.item_id.values
    probs = items_weights.weight.values
    
    recs = np.random.choice(items, size=n, p=probs, replace=False)
    
    return recs.tolist()

In [7]:
%%time

result2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result2.head(2)

items = data_train.item_id.unique()

result2['weighted_random_recommendation'] = result2['user_id'].apply(lambda x: weighted_random_recommendation(data_train, n=5))
result2.head(2)

CPU times: user 4.84 s, sys: 276 ms, total: 5.12 s
Wall time: 9.25 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[999779, 1003188, 12487888, 14106445, 10142833]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534178, 858373, 1132770, 1021164]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
result = pd.read_csv('../Recommender/webinar_2/predictions_basic.csv', converters={'random_recommendation': eval,
                                                                      'popular_recommendation': eval,
                                                                      'itemitem': eval,
                                                                      'cosine': eval,
                                                                      'tfidf': eval,
                                                                      'own_purchases': eval})

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popular, n=5))
result['actual'] = result2['actual']

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1265481, 5995177, 831470, 1726294, 38552]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1012701, 949991, 1040145, 880228, 874972]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2266886, 15563625, 6978868, 5653979, 833545]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5995830, 13512063, 12385320, 1077048, 10342382]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [10]:
def precision_at_k_(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [11]:
result.columns[2:]

Index(['random_recommendation', 'popular_recommendation',
       'weighted_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases'],
      dtype='object')

In [12]:
precisions = []
models = []
for column in result.columns[2:]:
    precision = result.apply(lambda x: precision_at_k_(x[column], x['actual']), axis=1).mean()
    precisions.append(precision)
    models.append(column)

metrics = pd.DataFrame(data={'models': models, 'presision@5': precisions}, columns=['models', 'presision@5'])
metrics

,models,presision@5
0,random_recommendation,0.000294
1,popular_recommendation,0.155240
2,weighted_random_recommendation,0.022429
3,itemitem,0.033595
4,cosine,0.035260
5,tfidf,0.036141
6,own_purchases,0.179693


Самый лучший результат показал алгоритм, рекомендующий собственный покупки пользователя, на втором месте по метрике алгоритм на основе популярных рекомендаций

### Задание 3. Улучшение бейзлайнов и ItemItem

- Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?

In [16]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)

    return recs.tolist()

def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""

    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)

    recs = popular.head(n).item_id

    return recs.tolist()

In [24]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000_items = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

result['top5000_random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(top_5000_items, n=5))

result['top5000_popularity_recommendation'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train.loc[data_train.item_id.isin(top_5000_items)], n=5))

result['top5000_weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(data_train.loc[data_train.item_id.isin(top_5000_items)], n=5))

result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases,top5000_random_recommendation,top5000_popularity_recommendation,top5000_weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1265481, 5995177, 831470, 1726294, 38552]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1012701, 949991, 1040145, 880228, 874972]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[1051323, 1054402, 1029688, 1089021, 905072]","[6534178, 6533889, 1029743, 6534166, 1082185]","[924352, 1103618, 6534178, 1082185, 957741]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2266886, 15563625, 6978868, 5653979, 833545]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5995830, 13512063, 12385320, 1077048, 10342382]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[1041157, 1056746, 961979, 1021562, 1092026]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1097001, 878996, 910032, 1101010, 923208]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8068255, 8019650, 880921, 12132277, 5566139]","[6534178, 6533889, 1029743, 6534166, 1082185]","[998373, 1118833, 10254131, 1004555, 1127831]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[976199, 923670, 940766, 982360, 1003487]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9884698, 15831255, 954077, 6534178, 1090701]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[7168388, 7102714, 8204289, 6904877, 1125491]","[6534178, 6533889, 1029743, 6534166, 1082185]","[828525, 841220, 1127624, 885863, 12517450]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]","[978906, 1083328, 897125, 1003178, 865174]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1129321, 879755, 918638, 1126899, 852856]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[9487360, 973124, 1066829, 13512863, 2620410]","[6534178, 6533889, 1029743, 6534166, 1082185]","[948650, 1010573, 989251, 887225, 13381928]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]","[1029479, 917494, 1024032, 973042, 1011876]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1100273, 6602327, 6534178, 973916, 1017195]"


In [26]:
columns = ['top5000_random_recommendation',
           'top5000_popularity_recommendation',
           'top5000_weighted_random_recommendation']
precisions = []
models = []
for column in columns:
    precision = result.apply(lambda x: precision_at_k_(x[column], x['actual']), axis=1).mean()
    precisions.append(precision)
    models.append(column)

metrics_top5000 = pd.DataFrame(data={'models': models, 'presision@5': precisions}, columns=['models', 'presision@5'])
metrics_top5000

,models,presision@5
0,top5000_random_recommendation,0.005485
1,top5000_popularity_recommendation,0.155240
2,top5000_weighted_random_recommendation,0.038100


In [27]:
metrics

,models,presision@5
0,random_recommendation,0.000294
1,popular_recommendation,0.155240
2,weighted_random_recommendation,0.022429
3,itemitem,0.033595
4,cosine,0.035260
5,tfidf,0.036141
6,own_purchases,0.179693


Если обучать на топ-5000 товаров, можно улучшить бейзлайны, кроме популярных - там результат такой же, потому что мы также берем помулярные, то есть, те же самые товары